# WeatherPy 
UCB Extension HW6
written by A. Lam

# Setup

In [15]:
# Modules
# Other modules are imported within the functions in which they are relevant
import pandas as pd
import matplotlib.pyplot as plt

# Generate Coordinates

In [2]:
def rand_coord():
    import random
    # Lat ranges from degrees [-90, 90] (N,S)
    lat = round(180*random.random()-90,2) 
    # Long ranges from degrees [-180, 180] (E,W)
    long = round(360*random.random()-180,2)
    return [lat,long]

# Convert Coordinates to City, Country

In [3]:
def coord2city(coords):
    # assume coords is a 2 item list of form: [lat,long]
    from citipy import citipy
    # extracts city name
    city = citipy.nearest_city(coords[0],coords[1]).city_name
    # extracts country code
    country = citipy.nearest_city(coords[0],coords[1]).country_code
    out = [city, country]
    return out

# API Call
API call is broken into three functions:
1. Call rand_coord and coord2city to generate an API call and touch endpoint
2. Check for a valid response, if not, run another API call
3. Output valid API call response to dataframe

In [4]:
def rand_city_API():
    # Modules
    import requests as req
    import json
    import OpenWeatherMap_API as owm
    
    # generate city and log coords, name, and country
    coords = rand_coord()
    place = coord2city(coords)
    
    # Keep variables for output
    city = place[0]
    country_code = place[1]
    
    city_id_q = '?q=' + city + ',' + country_code
    # API query structure
    base_url = 'http://api.openweathermap.org/data/2.5/weather'
    api_key = '&appid=' + owm.avl
    units = '&units=imperial'
    # construct query URL
    url = base_url+city_id_q+units+api_key
    # make API call
    response = req.get(url).json()
    
    #print(json.dumps(response,indent=2,sort_keys=True))
    
    return response

In [5]:
def response2df(response):
    import pandas as pd
    # assume input is a good API call to OpenWeatherMap API
    
    # output variables
    city = response['name']
    country_code = response['sys']['country']
    humidity = response['main']['humidity']
    temperature = response['main']['temp']
    cloudiness = response['clouds']['all']
    windspeed = response['wind']['speed']
    latitude = response['coord']['lat']
    longitude = response['coord']['lon']
        
    # Create dataframe of output variables
    output_order = ['City', 'Country Code', 'Latitude',
        'Longitude', 'Temperature', 'Humidity',
        'Cloudiness', 'Wind Speed']
    out = pd.DataFrame({
        'City' : city,
        'Country Code' : country_code,
        'Latitude' : latitude,
        'Longitude' : longitude,
        'Temperature' : temperature,
        'Humidity' : humidity,
        'Cloudiness' : cloudiness,
        'Wind Speed' : windspeed,
    }, index = [0])
    
    return out[output_order]

In [6]:
def rand_city_data():
    is_valid = False
    while is_valid == False:
        response = rand_city_API()
        if response['cod'] == 200:
            df = response2df(response)
            is_valid = True
        else:
            is_valid = False
    return df

In [13]:
# Generate Data Set
n_cities = 100 
data = rand_city_data() # initial df to append to
for i in range(n_cities - 1): # subtract 1 to accomodate for initial df
    data = data.append(rand_city_data(), ignore_index=True)

# Check if unique cities meets criteria
# if not, drop duplicate rows and replace with new data
while len((data['City']+data['Country Code']).unique()) != n_cities:
    print(str(len((data['City']+data['Country Code']).unique())) + 
          ' unique cities out of ' + str(n_cities) + ' requested.')
    data.drop_duplicates(keep = 'first', inplace = True)
    for i in range(n_cities - len((data['City']+data['Country Code']).unique())): 
        data = data.append(rand_city_data(), ignore_index=True)

# Print Data Set to .csv
data.to_csv('weather_data_avl.csv', index = False)

76 unique cities out of 100 requested.
88 unique cities out of 100 requested.
95 unique cities out of 100 requested.
97 unique cities out of 100 requested.
98 unique cities out of 100 requested.
98 unique cities out of 100 requested.
99 unique cities out of 100 requested.


In [14]:
data.head()

,City,Country Code,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Pangnirtung,CA,66.15,-65.71,3.55,88,56,6.64
1,Busselton,AU,-33.65,115.33,72.63,88,12,18.72
2,Quatre Cocos,MU,-20.21,57.76,84.20,74,40,8.05
3,Beringovskiy,RU,63.05,179.32,1.26,100,8,27.22
4,Hermanus,ZA,-34.42,19.23,59.94,77,44,3.85


In [ ]:
# Plot scatterplot of coordinates for coverage
plt.plot(data['Longitude'],data['Latitude'],'.')
plt.xlim([-180,180])
plt.ylim([-90,90])
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title

In [ ]:
# Plot Temperature vs. Latitude

In [ ]:
# Plot Humidity vs. Latitude

In [ ]:
# Plot Cloudiness vs. Latitude

In [ ]:
# Plot Wind Speed vs. Latitude

# Test Code

In [ ]:
# Test functions to generate cities
test = rand_coord()
print(test)
print(coord2city(test))
print(coord2city(rand_coord()))
coord2city(rand_coord())[0]

rand_city_API()['cod']


In [ ]:
for i in range(100):
    print(coord2city(rand_coord()))